# Residential Real Estate Project 1: Simple Linear Regression

In this notebook we will use data on house sales we downloaded from project 0 to predict house prices using simple (one input) linear regression. You will:
* Use python functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself to finish Project 1. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load house sales data

Dataset is from house sales in Santa Clara County, Palo Alto. 

In [3]:
df = pd.read_csv('merged.csv')

In [8]:
df.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,March-13-2015,Single Family Residential,1157 S Stelling Rd,CUPERTINO,CA,95014.0,1500000.0,3.0,2.0,...,Sold,NaN,NaN,http://www.redfin.com/CA/Cupertino/1157-S-Stel...,MLSListings,ML81448134,N,Y,37.303816,-122.041727
1,PAST SALE,April-17-2018,Single Family Residential,10340 Las Ondas Way,CUPERTINO,CA,95014.0,2798000.0,4.0,2.5,...,Sold,NaN,NaN,http://www.redfin.com/CA/Cupertino/10340-Las-O...,MLSListings,ML81695379,N,Y,37.317957,-122.024231
2,PAST SALE,February-4-2019,Single Family Residential,1035 W Homestead Rd,SUNNYVALE,CA,94087.0,2200000.0,4.0,3.0,...,Sold,NaN,NaN,http://www.redfin.com/CA/Sunnyvale/1035-W-Home...,MLSListings,ML81733182,N,Y,37.337792,-122.056622
3,PAST SALE,July-9-2018,Townhouse,11030 Firethorne Dr,CUPERTINO,CA,95014.0,1307000.0,2.0,2.5,...,Sold,NaN,NaN,http://www.redfin.com/CA/Cupertino/11030-Firet...,MLSListings,ML81708624,N,Y,37.338270,-122.032713
4,PAST SALE,July-12-2019,Single Family Residential,10590 S Tantau Ave,CUPERTINO,CA,95014.0,2090000.0,3.0,3.0,...,Sold,NaN,NaN,http://www.redfin.com/CA/Cupertino/10590-S-Tan...,MLSListings,ML81753390,N,Y,37.314344,-122.007310


check how many houses' sales data you have in the dataset. 

In [30]:
len(df.index)

2450

In [31]:
# We have 2450 data in this dataset, to count the unique house's number, using the ADDRESS column.

In [32]:
df['ADDRESS'].nunique()

2425

check what columns are there in this data set.

In [35]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

# Split data into training and testing

There are many ways to split data into training/validation/test sets. One way is to use random split and another common way is to use train_test_split in sklearn. To start with, you can do a 80/20 split. 

In [37]:
y = df.PRICE

In [40]:
X = df.drop('PRICE',axis=1)

In [96]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

Check how many hosues are in the training dataset and how many houses are in the test dataset.

In [198]:
print('There are %d houses in training dataset'%len(X_train))
print('There are %d houses in test dataset'%len(y_test))

There are 1960 houses in training dataset
There are 490 houses in test dataset


# Useful python summary functions

In order to make use of the closed form solution as well as take advantage of python's built in functions we will review some important ones. In particular:
* computing the sum of one column in dataframe.
* computing the arithmetic average (mean) of one column in dataframe.
* multiplying column by constants
* multiplying column by other column

In [61]:
# Let's compute the mean of the House Prices in Palo Alto in 2 different ways.
# extract the price column of the sales dataframe
# first method to calcualte the average price: recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
# notice there are houses with price nan. you need to figure out a way to exclude those houses.
# your code
price = df['PRICE']
price = price.dropna()
pravg1 = price.sum()/len(price.index)
print('The average price via method 1:',pravg1)


# second method to calcuate the average price, the .mean() function
pravg2 = price.mean()
# print results of both methods, they should be exactly the same
print('The average price via method 1:',pravg2)

The average price via method 1: 1044735.0284334763
The average price via method 1: 1044735.0284334763


As you see, we get the same answer both ways.

In [72]:
price_squared =df['PRICE'] * df['PRICE']

9709856654632820.0

In [73]:
# if we want to multiply every price by 0.5 and output it to variable half_prices
half_prices = 0.5 * price
# Let's compute the sum of squares of price. First multiply two columns of the same length elementwise with *
price_squared =df['PRICE'] * df['PRICE']
# price_squared is a variable of the squares and we want to add them up and save as sum_prices_squared.
sum_prices_squared = price_squared.sum()
# print the result
print("The sum of price squared is:",sum_prices_squared)

The sum of price squared is: 9709856654632820.0


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function

Armed with these python functions we can use the closed form solution to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.
Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [178]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    sum_input_feature = input_feature.sum()
    sum_output = output.sum()
    
    # compute the product of the output and the input_feature and its sum
    product = output * input_feature
    sum_product = product.sum()
    
    # compute the squared value of the input_feature and its sum
    squared_input_feature = input_feature * input_feature
    sum_squared_input_feature = squared_input_feature.sum()
    
    # derive the formula for the slope
    slope = sum_product/sum_squared_input_feature
    
    # derive the formula for the intercept
    intercept = sum_output - slope * sum_input_feature
    
    return (intercept, slope)

In [ ]:
## The above function does not work well. Therefore, I wrote my own function as follows:

In [14]:
def simple_linear_regression(input_feature, output):
    # Calculate the average of the input_feature and output
    mean_input_feature = input_feature.mean()
    mean_output = output.mean()
    
    # Calcuate the difference between the xi and intput_feature's average, between yi and output's average.
    # Calculate its product and sum
    xi_xmean = input_feature-mean_input_feature
    yi_ymean = output - mean_output
    product = xi_xmean * yi_ymean
    sum_product = product.sum()
    
    # Calculate the sum of the difference between xi and intput_feature's average and its sum.
    xi_xmean_squared = xi_xmean * xi_xmean
    xi_xmean_squared_sum = xi_xmean_squared.sum()
    
    #derive the formula for the slope
    slope = xi_xmean_squared_sum/sum_product
    
    #derive the formula for the intercept
    intercept = mean_output - slope * mean_input_feature
    
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1*input_feature then we know both our slope and intercept should be 1

In [15]:
test_feature = np.array(range(5))
test_output = 1 + 1*test_feature
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print ("Intercept: " + str(test_intercept))
print ("Slope: " + str(test_slope))

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on square feet. Rembember that we train on train_data!

In [89]:
y = df.PRICE
X = df['SQUARE FEET']

In [98]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [187]:
(sqft_intercept,sqft_slope) = simple_linear_regression(X_train,y_train)

In [188]:
print ("Intercept: " + str(sqft_intercept))
print ("Slope: " + str(sqft_slope))

Intercept: 1014940.9653516368
Slope: 0.0029433340243944705


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using np.array it's easy to multiply an np array by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [110]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = slope * input_feature + intercept
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2000 squarefeet according to the squarefeet model we estiamted above

In [114]:
my_house_sqft = 2000
estimated_price = get_regression_predictions(2000,1014940.9653516368,0.0029433340243944705)
print ("The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2000 squarefeet is $1014946.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output.

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [67]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    prediction = input_feature * slope + intercept
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = prediction - output
    # square the residuals and add them up
    residuals_sum = residuals.sum()
    RSS = residuals_sum * residuals_sum
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [171]:
test_feature = np.array(range(5))
test_output = 1 + 1*test_feature
(sqft_intercept,sqft_slope) = simple_linear_regression(X_train,y_train)

In [172]:
get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)

0

According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?

In [175]:
rss_prices_on_sqft = get_residual_sum_of_squares(X_train, y_train,sqft_intercept,sqft_slope)
print ('The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 353099094292777.2


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [176]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = (output-intercept)/slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

In [184]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(800000,sqft_intercept,sqft_slope)
print ("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $800000.00 is -297266


In [ ]:
## The intercept is above 800000, therefore, the minimum price for a house is 1014940.9653516368, you cannot buy
## a house for 800000.

# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales dataframe. Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [189]:
y = df.PRICE
X = df.BEDS

In [190]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [192]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
(bed_intercept,bed_slope) = simple_linear_regression(X_train,y_train)
print( "Intercept: " + str(bed_intercept))
print ("Slope: " + str(bed_slope))

Intercept: 1060375.5764930001
Slope: 5.287700625540953e-06


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

Which model(square feet or bedrooms) has lowest RSS on TEST data? 

Think about why this maybe the case.

How about other features in the dataset?

In [195]:
# Compute RSS when using bedrooms on TEST data:
rss_prices_on_beds_test = get_residual_sum_of_squares(X_test, y_test,bed_intercept,bed_slope)
rss_prices_on_beds_test

605713526495984.8

In [196]:
y = df.PRICE
X = df['SQUARE FEET']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [197]:
# Compute RSS when using squarefeet on TEST data:
rss_prices_on_sqft_test = get_residual_sum_of_squares(X_test, y_test,sqft_intercept,sqft_slope)
rss_prices_on_sqft_test

68059630999786.62

In [ ]:
## The BEDROOMS has the lowest RSS compared to the squarefeet. This means the model is more accurate compared to 
## another. 

# New model I: Estimte Price from City.

In [5]:
df.CITY.unique()

array(['CUPERTINO', 'SUNNYVALE', 'Cupertino', 'SAN JOSE', 'San Jose',
       'Unincorporated', 'Sunnyvale', 'MOUNTAIN VIEW', 'Los Altos',
       'Palo Alto', 'Mountain View', 'LOS ALTOS', 'Los Altos Hills',
       'LOS ALTOS HILLS', 'PALO ALTO', 'East Palo Alto', 'STANFORD',
       'EAST PALO ALTO', 'Menlo Park', 'REDWOOD CITY', 'Redwood City',
       'San Carlos', 'Redwood Shores', nan, 'Half Moon Bay', 'WOODSIDE',
       'Woodside', 'HALF MOON BAY', 'LA HONDA', 'Emerald Hills',
       'La Honda', 'LOS GATOS', 'CAMPBELL', 'Santa Clara', 'SARATOGA',
       'MILPITAS', 'SANTA CLARA', 'Hopkinton'], dtype=object)

In [6]:
# I noticed that there are same cites but written in different forms. Therefore, I need to change them to the same 
# form using uppercase format.

In [45]:
df['CITY'] = df.CITY.str.upper()

In [46]:
df.CITY.unique()

array(['CUPERTINO', 'SUNNYVALE', 'SAN JOSE', 'UNINCORPORATED',
       'MOUNTAIN VIEW', 'LOS ALTOS', 'PALO ALTO', 'LOS ALTOS HILLS',
       'EAST PALO ALTO', 'STANFORD', 'MENLO PARK', 'REDWOOD CITY',
       'SAN CARLOS', 'REDWOOD SHORES', nan, 'HALF MOON BAY', 'WOODSIDE',
       'LA HONDA', 'EMERALD HILLS', 'LOS GATOS', 'CAMPBELL',
       'SANTA CLARA', 'SARATOGA', 'MILPITAS', 'HOPKINTON'], dtype=object)

In [47]:
y = df.PRICE
X = df.CITY

In [19]:
# Transform City into caterocial data

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
labelencoder_X = LabelEncoder()

In [51]:
X = labelencoder_X.fit_transform(X.astype(str))

In [59]:
pd.Series(X).unique()

array([ 1, 21, 17, 22, 12,  7, 13,  8,  2, 20, 10, 14, 16, 15, 24,  4, 23,
        6,  3,  9,  0, 18, 19, 11,  5])

In [ ]:
# There are 24 unique cities in this dataset(including null-value). Therefore, we have 24 nummbers representing each
# city.

In [56]:
# Split the data into training set and test set

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [64]:
# Perform linear regression

In [65]:
(city_intercept,city_slope) = simple_linear_regression(X_train,y_train)

In [66]:
print ("Intercept: " + str(city_intercept))
print ("Slope: " + str(city_slope))

Intercept: 995306.7981013919
Slope: -5.3693374041952084e-05


In [68]:
# Calculate the RSS using test set.

In [70]:
rss_prices_on_city = get_residual_sum_of_squares(X_test, y_test,city_intercept,city_slope)
print ('The RSS of predicting Prices based on City is : ' + str(rss_prices_on_city))

The RSS of predicting Prices based on City is : 8488714964729607.0


# New Model II: Estimate Prices from House type

In [74]:
df['PROPERTY TYPE'].unique()

array(['Single Family Residential', 'Townhouse', 'Condo/Co-op',
       'Multi-Family (2-4 Unit)', 'Mobile/Manufactured Home',
       'Multi-Family (5+ Unit)', 'Unknown', 'Vacant Land', 'Other'],
      dtype=object)

In [75]:
y = df.PRICE
X = df['PROPERTY TYPE']

In [76]:
# Transform the X data into caterogical data.

In [77]:
labelencoder_X = LabelEncoder()

In [78]:
X = labelencoder_X.fit_transform(X.astype(str))

In [79]:
pd.Series(X).unique()

array([5, 6, 0, 2, 1, 3, 7, 8, 4])

In [80]:
# There are 9 types of house in this dataset.
# Therefore, we have 9 numbers representing each different type of house.

In [81]:
# Split the dataset into training set and test size.

In [82]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [83]:
# Perform simple linear regression

In [84]:
(house_intercept,house_slope) = simple_linear_regression(X_train,y_train)

In [85]:
print ("Intercept: " + str(house_intercept))
print ("Slope: " + str(house_slope))

Intercept: 1049167.149763268
Slope: 2.4105048898925487e-05


In [86]:
# Calculate the SSR using test dataset.

In [88]:
rss_prices_on_housetype = get_residual_sum_of_squares(X_test, y_test,house_intercept,house_slope)
print ('The RSS of predicting Prices based on Housetype is : ' + str(rss_prices_on_housetype))

The RSS of predicting Prices based on Housetype is : 68251958390548.26


# Conclusion

In [89]:
# From the above models, we have the results as follows:
# SSR for bedrooms is 605713526495984.8.
# SSR for squarefeet is 68059630999786.62.
# SSR for city is 8488714964729607.0.
# SSR for housetype is 68251958390548.26.

In [90]:
# SSR for bedrooms is still the lowest. Meaning the simple regression line could fit the data better.
# To know how well the regression fit the data, we need to calculate the SST,SSE and also the R - squared.

In [ ]:
# Besides, what I have done is to consider one variable one time. In real life, we cannot consider only one variable 
# one time. We need to consider all the factors that may influence the results.
# Therefore, we need to perform the multivariate regression.